In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Dropout, Concatenate, Embedding, Flatten, Bidirectional, BatchNormalization, Activation, Multiply, Permute, RepeatVector, Lambda, GlobalAveragePooling1D
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
import os
import gc
import zipfile

# ==========================================
# 0. SETUP & UNZIP
# ==========================================
zip_filename = "wow.zip"
if os.path.exists(zip_filename):
    print(f"Found {zip_filename}. Extracting...")
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall(".")
    print("✅ Extraction complete.")
else:
    print(f"⚠️ {zip_filename} not found. Assuming files are already present.")

# ==========================================
# 1. CONFIG & PATHS
# ==========================================
BASE_DIR = "."
PATHS = {
    "climate": os.path.join(BASE_DIR, "iklim", "aa-combined.csv"),
    "soil_moisture": os.path.join(BASE_DIR, "tanah", "soilmoisture.csv"),
    "soil_static": os.path.join(BASE_DIR, "tanah", "soilstatic.csv"),
    "harvest": os.path.join(BASE_DIR, "tanaman", "waktupanen.csv"),
    "target": os.path.join(BASE_DIR, "tanaman", "biofarmaka.csv"),
    "output": os.path.join(BASE_DIR, "Biofarmaka_Prediction_2025.csv"),
    "plot_luas": os.path.join(BASE_DIR, "eval_luas_panen.png"),
    "plot_prod": os.path.join(BASE_DIR, "eval_produksi.png")
}

TRAIN_START = "2023-11-01"
TRAIN_END = "2024-12-31"
PREDICT_START = "2025-01-01"
PREDICT_END = "2025-10-31"

# HYPERTUNING: Increased Lookback Window to 90 days (Quarterly)
# This captures a fuller crop cycle context
LOOKBACK_WINDOW = 90

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# ==========================================
# 2. DATA LOADING & CLEANING
# ==========================================
def clean_province(prov):
    if pd.isna(prov): return "UNKNOWN"
    return str(prov).strip().upper()

print("Loading datasets...")

for key in ["climate", "soil_moisture", "soil_static", "harvest", "target"]:
    if not os.path.exists(PATHS[key]):
        print(f"❌ Warning: {key} file not found at {PATHS[key]}")

# Load climate
df_climate = pd.read_csv(PATHS["climate"])
df_climate['TANGGAL'] = pd.to_datetime(df_climate['TANGGAL'], dayfirst=True, errors='coerce')
df_climate = df_climate.dropna(subset=['TANGGAL'])
df_climate['Provinsi'] = df_climate['Provinsi'].apply(clean_province)
df_climate = df_climate.groupby(['TANGGAL', 'Provinsi']).mean(numeric_only=True).reset_index()

# Load soil moisture
df_moisture = pd.read_csv(PATHS["soil_moisture"])
df_moisture['Date'] = pd.to_datetime(df_moisture['Date'], errors='coerce')
df_moisture['Provinsi'] = df_moisture['Provinsi'].apply(clean_province)
df_moisture.rename(columns={'Date': 'TANGGAL', 'Soil_Moisture_%': 'Soil_Moisture'}, inplace=True)

# Load soil static
df_soil_static = pd.read_csv(PATHS["soil_static"])
df_soil_static['Provinsi'] = df_soil_static['Provinsi'].apply(clean_province)

# Load harvest info
df_harvest = pd.read_csv(PATHS["harvest"])
df_harvest['Avg_Harvest_Days'] = (df_harvest['Waktu Panen Minimum (Hari)'] + df_harvest['Waktu Panen Maksimum (Hari)']) / 2
df_harvest = df_harvest[['Jenis Tanaman', 'Avg_Harvest_Days']]

# Load target
df_target = pd.read_csv(PATHS["target"])
df_target['Date'] = pd.to_datetime(df_target['Date'], errors='coerce')
df_target['Provinsi'] = df_target['Provinsi'].apply(clean_province)
df_target.rename(columns={'Date': 'TANGGAL'}, inplace=True)

# ==========================================
# 3. MERGE & FEATURES
# ==========================================
print("Merging data...")

all_dates = pd.date_range(start=TRAIN_START, end=PREDICT_END, freq='D')
unique_provinces = df_target['Provinsi'].unique()
unique_crops = df_target['Jenis Tanaman'].unique()

index = pd.MultiIndex.from_product([all_dates, unique_provinces, unique_crops],
                                   names=['TANGGAL', 'Provinsi', 'Jenis Tanaman'])
df_master = pd.DataFrame(index=index).reset_index()

# merge dynamic features and forward/backfill per province to fill gaps
df_features = pd.merge(df_climate, df_moisture, on=['TANGGAL', 'Provinsi'], how='outer')
df_features = df_features.sort_values('TANGGAL')
df_features = df_features.groupby('Provinsi').apply(lambda x: x.ffill().bfill()).reset_index(drop=True)

df_master = pd.merge(df_master, df_features, on=['TANGGAL', 'Provinsi'], how='left')
df_master = pd.merge(df_master, df_soil_static, on=['Provinsi'], how='left')
df_master = pd.merge(df_master, df_harvest, on=['Jenis Tanaman'], how='left')
df_master = pd.merge(df_master, df_target, on=['TANGGAL', 'Provinsi', 'Jenis Tanaman'], how='left')

# fill static cols
static_cols = ['pH', 'Clay_%', 'Sand_%', 'Organic_Carbon_g/kg', 'Avg_Harvest_Days']
for col in static_cols:
    if col in df_master.columns:
        df_master[col] = df_master[col].fillna(df_master[col].mean())

df_master = df_master.sort_values(['Provinsi', 'Jenis Tanaman', 'TANGGAL'])

# ==========================================
# 4. PREPROCESSING
# ==========================================
print("Preprocessing features...")

dynamic_features = ['TN', 'TX', 'TAVG', 'RH_AVG', 'RR', 'SS', 'Soil_Moisture']
static_numeric_features = ['pH', 'Clay_%', 'Sand_%', 'Organic_Carbon_g/kg', 'Avg_Harvest_Days']
targets = ['Luas Panen', 'Produksi']

# Fill zeros
for col in dynamic_features:
    if col in df_master.columns:
        df_master[col] = df_master[col].fillna(0)
    else:
        df_master[col] = 0.0

for t in targets:
    if t not in df_master.columns:
        df_master[t] = 0.0
    df_master[t] = df_master[t].fillna(0.0).astype(float)

# LOG TRANSFORM
print("Applying Log1p Transform to Targets...")
df_master['Luas_log'] = np.log1p(df_master['Luas Panen'].astype(float))
df_master['Produksi_log'] = np.log1p(df_master['Produksi'].astype(float))

# HYPERTUNING: Use StandardScaler for dynamic features to handle weather outliers better than MinMax
scaler_dynamic = StandardScaler()
scaler_static = MinMaxScaler()

df_master[dynamic_features] = scaler_dynamic.fit_transform(df_master[dynamic_features])
df_master[static_numeric_features] = scaler_static.fit_transform(df_master[static_numeric_features])

# Label Encode
le_prov = LabelEncoder()
df_master['Prov_ID'] = le_prov.fit_transform(df_master['Provinsi'])
le_crop = LabelEncoder()
df_master['Crop_ID'] = le_crop.fit_transform(df_master['Jenis Tanaman'])

# ==========================================
# 5. CREATE SEQUENCES
# ==========================================
print("Generating sequences...")

train_mask = (df_master['TANGGAL'] >= TRAIN_START) & (df_master['TANGGAL'] <= TRAIN_END)
predict_mask = (df_master['TANGGAL'] >= PREDICT_START) & (df_master['TANGGAL'] <= PREDICT_END)

df_train_raw = df_master[train_mask].copy()
df_predict_raw = df_master[predict_mask].copy()

# Fit target scaler (StandardScaler on Log Targets) on TRAIN ONLY
log_targets_train = df_train_raw[['Luas_log', 'Produksi_log']].values
scaler_target = StandardScaler()
scaler_target.fit(log_targets_train)

def create_sequences(df, is_training=True):
    X_dynamic, X_static, X_prov, X_crop, y, meta = [], [], [], [], [], []
    grouped = df.groupby(['Provinsi', 'Jenis Tanaman'])

    for (prov, crop), group in grouped:
        group = group.sort_values('TANGGAL')
        if len(group) <= LOOKBACK_WINDOW:
            continue

        dyn_data = group[dynamic_features].values
        stat_data = group[static_numeric_features].values
        prov_id = int(group['Prov_ID'].values[0])
        crop_id = int(group['Crop_ID'].values[0])
        dates = group['TANGGAL'].values

        if is_training:
            target_log = group[['Luas_log', 'Produksi_log']].values
            target_scaled = scaler_target.transform(target_log)

        for i in range(LOOKBACK_WINDOW, len(group)):
            X_dynamic.append(dyn_data[i-LOOKBACK_WINDOW:i])
            X_static.append(stat_data[i])
            X_prov.append(prov_id)
            X_crop.append(crop_id)
            meta.append([dates[i], prov, crop])
            if is_training:
                y.append(target_scaled[i])

    if len(X_dynamic) == 0:
        return (np.empty((0, LOOKBACK_WINDOW, len(dynamic_features))), np.empty((0, len(static_numeric_features))), np.empty((0, 1)), np.empty((0, 1)), np.empty((0, 2)), meta)

    X_dyn = np.array(X_dynamic, dtype=np.float32)
    X_stat = np.array(X_static, dtype=np.float32)
    X_prov = np.array(X_prov, dtype=np.int32).reshape(-1, 1)
    X_crop = np.array(X_crop, dtype=np.int32).reshape(-1, 1)
    y_arr = np.array(y, dtype=np.float32) if is_training else None

    return X_dyn, X_stat, X_prov, X_crop, y_arr, meta

X_dyn_train, X_stat_train, X_prov_train, X_crop_train, y_train, _ = create_sequences(df_train_raw, is_training=True)

# Prediction buffer
last_days_per_group = df_train_raw.groupby(['Provinsi', 'Jenis Tanaman']).tail(LOOKBACK_WINDOW)
df_predict_buffered = pd.concat([last_days_per_group, df_predict_raw]).sort_values(['Provinsi', 'Jenis Tanaman', 'TANGGAL'])
X_dyn_pred, X_stat_pred, X_prov_pred, X_crop_pred, _, meta_pred = create_sequences(df_predict_buffered, is_training=False)

print("Training Sequences:", X_dyn_train.shape)

# ==========================================
# 6. MODEL ARCHITECTURE (SOTA UPGRADE)
# ==========================================
def attention_block(inputs):
    # inputs.shape = (batch_size, time_steps, input_dim)
    input_dim = int(inputs.shape[2])
    a = Permute((2, 1))(inputs)
    a = Dense(inputs.shape[1], activation='softmax')(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    output_attention_mul = Multiply()([inputs, a_probs])
    return output_attention_mul

def build_sota_model(num_provs, num_crops):
    # --- Branch 1: Dynamic (Bi-LSTM + Attention) ---
    input_dyn = Input(shape=(LOOKBACK_WINDOW, len(dynamic_features)), name='dynamic_input')

    x = Bidirectional(LSTM(128, return_sequences=True))(input_dyn)
    x = BatchNormalization()(x)
    x = Activation('swish')(x) # Swish is often better than ReLU for deep regressors
    x = Dropout(0.3)(x)

    # Self-Attention Mechanism to weigh important days
    x = attention_block(x)
    x = GlobalAveragePooling1D()(x) # Collapse time dimension weighted by attention

    # --- Branch 2: Static ---
    input_stat = Input(shape=(len(static_numeric_features),), name='static_input')
    x_stat = Dense(64)(input_stat)
    x_stat = BatchNormalization()(x_stat)
    x_stat = Activation('swish')(x_stat)

    # --- Branch 3: Embeddings (Increased Capacity) ---
    # Larger dim helps memorize high-production provinces
    input_prov = Input(shape=(1,), name='prov_input')
    emb_prov = Embedding(input_dim=num_provs, output_dim=24)(input_prov)
    emb_prov = Flatten()(emb_prov)

    input_crop = Input(shape=(1,), name='crop_input')
    emb_crop = Embedding(input_dim=num_crops, output_dim=16)(input_crop)
    emb_crop = Flatten()(emb_crop)

    # --- Fusion ---
    combined = Concatenate()([x, x_stat, emb_prov, emb_crop])

    # --- Deep Interaction Block ---
    z = Dense(256)(combined)
    z = BatchNormalization()(z)
    z = Activation('swish')(z)
    z = Dropout(0.3)(z)

    z = Dense(128)(z)
    z = BatchNormalization()(z)
    z = Activation('swish')(z)

    # --- SEPARATE TOWERS (Multi-Task Learning) ---
    # Tower 1: Luas Panen
    t1 = Dense(64, activation='swish')(z)
    t1 = Dense(32, activation='swish')(t1)
    out1 = Dense(1, name='out_luas')(t1)

    # Tower 2: Produksi
    t2 = Dense(64, activation='swish')(z)
    t2 = Dense(32, activation='swish')(t2)
    out2 = Dense(1, name='out_prod')(t2)

    # Recombine for simple compatibility with array target
    output = Concatenate(name='final_output')([out1, out2])

    model = Model(inputs=[input_dyn, input_stat, input_prov, input_crop], outputs=output)

    # Nadam is Adam with Nesterov momentum, often converges faster/better
    model.compile(optimizer=Nadam(learning_rate=2e-3), loss='mse', metrics=['mae'])
    return model

model = build_sota_model(num_provs=len(unique_provinces), num_crops=len(unique_crops))
model.summary()

# ==========================================
# 7. TRAINING
# ==========================================
print("Training model...")

if X_dyn_train.shape[0] == 0:
    raise RuntimeError("No training data.")

X_dyn_tr, X_dyn_val, X_stat_tr, X_stat_val, X_p_tr, X_p_val, X_c_tr, X_c_val, y_tr, y_val = train_test_split(
    X_dyn_train, X_stat_train, X_prov_train, X_crop_train, y_train, test_size=0.15, random_state=42
)

lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-6, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True, verbose=1)

history = model.fit(
    x=[X_dyn_tr, X_stat_tr, X_p_tr, X_c_tr],
    y=y_tr,
    validation_data=([X_dyn_val, X_stat_val, X_p_val, X_c_val], y_val),
    epochs=80, # More epochs, relying on early stopping
    batch_size=128,
    callbacks=[early_stop, lr_scheduler],
    verbose=1
)

# ==========================================
# 7.5 EVALUATION
# ==========================================
print("\n" + "="*40)
print("EVALUATION ON VALIDATION SET")
print("="*40)

y_pred_scaled = model.predict([X_dyn_val, X_stat_val, X_p_val, X_c_val], batch_size=512)

# Inverse transform
y_val_log = scaler_target.inverse_transform(y_val)
y_pred_log = scaler_target.inverse_transform(y_pred_scaled)

y_val_real = np.expm1(y_val_log)
y_pred_real = np.expm1(y_pred_log)
y_pred_real = np.clip(y_pred_real, 0, None)

def masked_mape(a, f, min_denominator=1.0):
    denom = np.maximum(np.abs(a), min_denominator)
    return 100.0 * np.mean(np.abs((a - f) / denom))

targets_list = ['Luas Panen', 'Produksi']
for i, target_name in enumerate(targets_list):
    actual = y_val_real[:, i]
    predicted = y_pred_real[:, i]

    mae = mean_absolute_error(actual, predicted)
    rmse = np.sqrt(mean_squared_error(actual, predicted))
    r2 = r2_score(actual, predicted)
    mape_masked = masked_mape(actual, predicted, min_denominator=1.0)

    print(f"\nResults for {target_name}:")
    print(f"  MAE    : {mae:,.2f}")
    print(f"  RMSE   : {rmse:,.2f}")
    print(f"  MAPE* : {mape_masked:.2f}%")
    print(f"  R2     : {r2:.4f}")

    plt.figure(figsize=(8, 6))
    plt.scatter(actual, predicted, alpha=0.4, s=15, edgecolors='k', linewidth=0.1)
    min_val = float(min(actual.min(), predicted.min()))
    max_val = float(max(actual.max(), predicted.max()))
    plt.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2)
    plt.xlabel(f'Actual {target_name}')
    plt.ylabel(f'Predicted {target_name}')
    plt.title(f'{target_name}: Actual vs Predicted\nR2: {r2:.3f}')
    plt.grid(True, alpha=0.3)
    save_path = PATHS['plot_luas'] if i == 0 else PATHS['plot_prod']
    plt.savefig(save_path)
    plt.close()

# ==========================================
# 8. FINAL PREDICTION
# ==========================================
print("\n" + "="*40)
print("GENERATING FUTURE PREDICTIONS (2025)")
print("="*40)

if X_dyn_pred.shape[0] > 0:
    preds_scaled = model.predict([X_dyn_pred, X_stat_pred, X_prov_pred, X_crop_pred], batch_size=512)
    preds_log = scaler_target.inverse_transform(preds_scaled)
    preds_original = np.expm1(preds_log)
    preds_original = np.clip(preds_original, 0, None)

    df_result = pd.DataFrame(meta_pred, columns=['Date', 'Provinsi', 'Jenis Tanaman'])
    df_result['Luas Panen'] = preds_original[:, 0]
    df_result['Produksi'] = preds_original[:, 1]
    df_result['Date'] = pd.to_datetime(df_result['Date'])
    df_result = df_result.sort_values(['Date', 'Provinsi', 'Jenis Tanaman'])

    print(f"Saving to {PATHS['output']}...")
    df_result.to_csv(PATHS['output'], index=False)
    print("✅ Saved predictions.")
else:
    print("No prediction data available.")

print("✅ Process complete.")

Found wow.zip. Extracting...
✅ Extraction complete.
Num GPUs Available:  1
Loading datasets...
Merging data...


/tmp/ipython-input-4130838162.py:112: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_features = df_features.groupby('Provinsi').apply(lambda x: x.ffill().bfill()).reset_index(drop=True)


Preprocessing features...
Applying Log1p Transform to Targets...
Generating sequences...
Training Sequences: (192090, 90, 7)


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ dynamic_input       │ (None, 90, 7)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_4     │ (None, 90, 256)   │    139,264 │ dynamic_input[0]… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 90, 256)   │      1,024 │ bidirectional_4[… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 90, 256)   │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_9 (Dropout) │ (None, 90, 256)   │          0 │ activation[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ permute (Permute)   │ (None, 256, 90)   │          0 │ dropout_9[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 256, 90)   │      8,190 │ permute[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ static_input        │ (None, 5)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_vec       │ (None, 90, 256)   │          0 │ dense_9[0][0]     │
│ (Permute)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 64)        │        384 │ static_input[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ prov_input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ crop_input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 90, 256)   │          0 │ dropout_9[0][0],  │
│                     │                   │            │ attention_vec[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64)        │        256 │ dense_10[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_6         │ (None, 1, 24)     │        912 │ prov_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_7         │ (None, 1, 16)     │        240 │ crop_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 256)       │          0 │ multiply[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 64)        │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                 

 Total params: 297,856 (1.14 MB)

 Trainable params: 296,448 (1.13 MB)

 Non-trainable params: 1,408 (5.50 KB)

Training model...
Epoch 1/80
1276/1276 ━━━━━━━━━━━━━━━━━━━━ 49s 32ms/step - loss: 0.1625 - mae: 0.2854 - val_loss: 0.0295 - val_mae: 0.1284 - learning_rate: 0.0020
Epoch 2/80
1276/1276 ━━━━━━━━━━━━━━━━━━━━ 40s 31ms/step - loss: 0.0386 - mae: 0.1485 - val_loss: 0.0267 - val_mae: 0.1233 - learning_rate: 0.0020
Epoch 3/80
1276/1276 ━━━━━━━━━━━━━━━━━━━━ 40s 32ms/step - loss: 0.0321 - mae: 0.1354 - val_loss: 0.0251 - val_mae: 0.1186 - learning_rate: 0.0020
Epoch 4/80
1276/1276 ━━━━━━━━━━━━━━━━━━━━ 40s 31ms/step - loss: 0.0298 - mae: 0.1302 - val_loss: 0.0249 - val_mae: 0.1177 - learning_rate: 0.0020
Epoch 5/80
1276/1276 ━━━━━━━━━━━━━━━━━━━━ 39s 31ms/step - loss: 0.0285 - mae: 0.1270 - val_loss: 0.0252 - val_mae: 0.1184 - learning_rate: 0.0020
Epoch 6/80
1276/1276 ━━━━━━━━━━━━━━━━━━━━ 40s 31ms/step - loss: 0.0274 - mae: 0.1247 - val_loss: 0.0250 - val_mae: 0.1197 - learning_rate: 0.0020
Epoch 7/80
1276/1276 ━━━━━━━━━━━━━━━━━━━━ 40s 31ms/step - loss: 0.0269 - mae: 0.1230 - val_loss: 0.0241 - 